In [1]:
import requests
from datetime import date, timedelta
from time import sleep
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
"""
countries = {'ar':'Argentina', 'at':'Austria', 'au':'Australia', 'be':'Belgium',
       'bo':'Bolivia', 'br':'Brazil', 'ca':'Canada', 'ch':'Switzerland', 
       'cl':'Chile', 'co':'Columbia', 'cr':'CostaRica', 'cz':'CzechRepublic',
       'de':'Germany', 'dk':'Denmark', 'do':'DominicanRepublic', 'ec':'Ecuador', 
       'ee':'Estonia', 'es':'Spain', 'fi':'Finland', 'fr':'France', 
       'gb':'UnitedKingdom', 'global':'World', 'gr':'Greece', 'gt':'Guatemala',
       'hk':'HongKong', 'hn':'Honduras', 'hu':'Hungary', 'id':'Indonesia', 
       'ie':'Ireland', 'is':'Iceland', 'it':'Italy', 'jp':'Japan', 
       'lt':'Lithuania', 'lu':'Luxemborg', 'lv':'Latvia', 'mx':'Mexico', 
       'my':'Malaysia', 'nl':'Netherlands', 'no':'Norway', 'nz':'NewZealand', 
       'pa':'Panama', 'pe':'Peru', 'ph':'Philippines', 'pl':'Poland', 
       'pt':'Portugal', 'py':'Paraguay', 'se':'Sweden', 'sg':'Singapore', 
       'sk':'Slovakia', 'sv':'ElSalvador', 'tr':'Turkey', 'tw':'Taiwan', 'us':'USA', 'uy':'Uruguay'} 
"""

countries= {'at':'Austria', 'au':'Australia'}
new_countries= {'ae':'UnitedArabEmirates', 'bg':'Bulgaria', 'by':'Belarus', 'eg':'Egypt', 'il': 'Israel',
               'in': 'India', 'kr': 'Korea', 'kz':'Kazakhstan', 'ma': 'Marocco','ng': 'Nigeria','ni':'Nicaragua',
               'ro':'Romania','ru': 'Russia', 'sa': 'SaudiArabia', 'ua':'Ukraine', 'vn':'Vietnam', 'za':'SouthAfrica'}

In [3]:
country_codes=list(countries.keys())

In [4]:
processed_countries = []
processed_countries

[]

In [5]:
country_codes = [x for x in country_codes if x not in processed_countries]

In [6]:
def add_url(dates, base_url):
    url_list=[]
    for date in dates:
        c_string = base_url+date
        url_list.append(c_string)
    return url_list

def song_scrape(x,songs,country):
    pg = x
    pos=1
    song_lst = []
    for tr in songs.find("tbody").findAll("tr"):
        artist= tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist= artist.replace("by ","").strip()
  
        title= tr.find("td", {"class": "chart-table-track"}).find("strong").text
    
        streams = tr.find("td", {"class": "chart-table-streams"}).text
        streams = int(streams.replace(',',''))
 
        songid= tr.find("td", {"class": "chart-table-image"}).find("a").get("href")
        #songid= songid.split("track/")[1]
        songid_lst = songid.split("track/")
        songid = ''
        if len(songid_lst)>=2:
            songid = songid_lst[1]
    
        url_date= x.split("daily/")[1]
        song_lst.append([pos, title, artist, streams, songid, url_date, country])
        pos+=1
    return song_lst

#loop through urls to create array of all of our song info


def get_songs(countries, start_date, end_date):
    
    for country in tqdm(countries):
        final = []

        url = f"https://spotifycharts.com/regional/{country}/daily/"

        delta= end_date-start_date
        dates_=[]

        for i in range(delta.days+1):
            day = start_date+timedelta(days=i)
            day_string= day.strftime("%Y-%m-%d")
            dates_.append(day_string)

        url_list_=add_url(dates_,url)

        for i in tqdm(range(0, len(url_list_)), leave=False):
            
            songs_= None
            try:
                headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
                read_pg= requests.get(url_list_[i],headers=headers)
                sleep(2)
                #print(read_pg.text)
                soup= BeautifulSoup(read_pg.text, "html.parser")
                songs_= soup.find("table", {"class":"chart-table"})
            except:
                print(f"Error with url {url_list_[i]}. Waiting 2 hours.") 
                sleep(2*60*60) 
                i = i-1 #retry the same url
            
            if songs_ is not None:
                song_lst= song_scrape(url_list_[i],songs_,country)
                final= final + song_lst
            else:
                print(f"No chart for {url_list_[i]}.")
                
            i = i+1
                                            
        #convert to data frame with pandas for easier data manipulation

        final_df = pd.DataFrame(final, columns= ["Position", "Track Name", "Artist", "Streams", "URL", "Date", "Region"])

        #write to csv
        start_date_st= start_date.strftime('%m_%y')
        end_date_st= end_date.strftime('%m_%y')

        with open(f'sp_data_{country}_{start_date_st}-{end_date_st}.csv', 'w', encoding="utf-8") as f:
                final_df.to_csv(f, header= True, index=False)

In [7]:
start_date= date(2018, 1, 1)
end_date= date(2021, 6, 5)
get_songs(country_codes, start_date, end_date)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1252 [00:00<?, ?it/s]

  0%|          | 0/1252 [00:00<?, ?it/s]

In [8]:
print("That's all folks!!")

That's all folks!!
